# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Get non-repeated words
    vocab = set(text)
    
    # Create a dictionary {word: index}
    vocab_to_int = {word:i for i, word in enumerate(vocab)}
    
    # Create a dictionary {index: word}
    int_to_vocab = {i:word for i, word in enumerate(vocab)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_lookup_table = {
        '.': "||period||",
        ',': "||comma||",
        '"': "||quotation_mark||",
        ';': "||semicolon||",
        '!': "||exclamation_mark||",
        '?': "||question_mark||",
        '(': "||left_parenthesis||",
        ')': "||right_parenthesis||",
        '-': "||dash||",
        '\n': "||return||",
    }        
    return token_lookup_table

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # get number of targets we can make
    n_targets = len(words) - sequence_length
    # initialize feature and target 
    feature, target = [], []
    # loop through all targets we can make
    for i in range(n_targets):
        x = words[i : i+sequence_length]    # get some words from the given list
        y = words[i+sequence_length]        # get the next word to be the target
        feature.append(x)
        target.append(y)

    feature_tensor, target_tensor = torch.from_numpy(np.array(feature)), torch.from_numpy(np.array(target))
    # create data
    data = TensorDataset(feature_tensor, target_tensor)
    # create dataloader
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
     # return a dataloader
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 7,  8,  9, 10, 11],
        [41, 42, 43, 44, 45],
        [31, 32, 33, 34, 35],
        [24, 25, 26, 27, 28],
        [13, 14, 15, 16, 17],
        [35, 36, 37, 38, 39],
        [ 6,  7,  8,  9, 10],
        [30, 31, 32, 33, 34],
        [ 5,  6,  7,  8,  9],
        [23, 24, 25, 26, 27]])

torch.Size([10])
tensor([12, 46, 36, 29, 18, 40, 11, 35, 10, 28])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [82]:
import torch.nn as nn

"""
took from Sentiment_RNN_Solution.ipynb
"""
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()

        # set class variables
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.fc1 = nn.Linear(hidden_dim, hidden_dim*2)
        self.batchnorm_hidden1 = nn.BatchNorm1d(num_features=hidden_dim*2)
        self.fc2 = nn.Linear(hidden_dim*2, hidden_dim)
        self.batchnorm_hidden2 = nn.BatchNorm1d(num_features=hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        out = out.contiguous().view(-1, self.hidden_dim)

        out = self.fc1(out)
        out = self.batchnorm_hidden1(out)
        out = self.fc2(out)
        out = self.batchnorm_hidden2(out)
        out = self.fc3(out)

        # reshape to be batch_size first
        batch_size = nn_input.size(0)
        out = out[-batch_size:]

        # return last sigmoid output and hidden state    
        return out, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''    
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden
        
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [83]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    if (train_on_gpu):
        inp, target = inp.cuda(), target.cuda()

    # Creating new variables for the hidden state,
    # otherwise, we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])

    # perform forward pass
    out, hidden = rnn(inp, hidden)

    ## perform backpropagation and optimization
    optimizer.zero_grad()
    loss = criterion(out, target)
    loss.backward()

    # prevents the exploding gradient problem
    nn.utils.clip_grad_norm_(rnn.parameters(), 1)

    # optimize the gradients
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [84]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [95]:
# Data params
# Sequence Length
sequence_length = 8  # of words in a sequence
# Batch Size
batch_size = 256

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [100]:
# Training parameters
# Number of Epochs
num_epochs = 200
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 64
# Hidden Dimension
hidden_dim = 64
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 100

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [101]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 200 epoch(s)...
Epoch:    1/200   Loss: 9.899205379486084
Epoch:    1/200   Loss: 8.977830567359923
Epoch:    1/200   Loss: 6.919186291694641
Epoch:    1/200   Loss: 5.845493769645691
Epoch:    1/200   Loss: 5.757588353157043
Epoch:    1/200   Loss: 5.786243281364441
Epoch:    1/200   Loss: 5.764702110290528
Epoch:    1/200   Loss: 5.756225914955139
Epoch:    1/200   Loss: 5.767531013488769
Epoch:    1/200   Loss: 5.759362840652466
Epoch:    1/200   Loss: 5.764714212417602
Epoch:    1/200   Loss: 5.7369367170333865
Epoch:    1/200   Loss: 5.733419466018677
Epoch:    1/200   Loss: 5.7225244522094725
Epoch:    1/200   Loss: 5.758004570007325
Epoch:    1/200   Loss: 5.7400441122055055
Epoch:    1/200   Loss: 5.759667196273804
Epoch:    1/200   Loss: 5.71513491153717
Epoch:    1/200   Loss: 5.659371671676635
Epoch:    1/200   Loss: 5.710945301055908
Epoch:    1/200   Loss: 5.7286837911605835
Epoch:    1/200   Loss: 5.741763944625855
Epoch:    1/200   Loss: 5.764271049499512
Ep

Epoch:    6/200   Loss: 5.6933038568496706
Epoch:    6/200   Loss: 5.666146488189697
Epoch:    6/200   Loss: 5.671510047912598
Epoch:    6/200   Loss: 5.6945803308486935
Epoch:    6/200   Loss: 5.654831395149231
Epoch:    6/200   Loss: 5.678479909896851
Epoch:    6/200   Loss: 5.652579588890076
Epoch:    6/200   Loss: 5.678024845123291
Epoch:    6/200   Loss: 5.694584436416626
Epoch:    7/200   Loss: 5.638324957826863
Epoch:    7/200   Loss: 5.686337494850159
Epoch:    7/200   Loss: 5.639008455276489
Epoch:    7/200   Loss: 5.68415922164917
Epoch:    7/200   Loss: 5.6516822147369385
Epoch:    7/200   Loss: 5.6274777603149415
Epoch:    7/200   Loss: 5.6788491439819335
Epoch:    7/200   Loss: 5.640529890060424
Epoch:    7/200   Loss: 5.692067003250122
Epoch:    7/200   Loss: 5.677824773788452
Epoch:    7/200   Loss: 5.657211532592774
Epoch:    7/200   Loss: 5.676866292953491
Epoch:    7/200   Loss: 5.660528354644775
Epoch:    7/200   Loss: 5.655565347671509
Epoch:    7/200   Loss: 5.6547

Epoch:   12/200   Loss: 5.654249548912048
Epoch:   12/200   Loss: 5.683975758552552
Epoch:   12/200   Loss: 5.673229432106018
Epoch:   12/200   Loss: 5.6505513048172
Epoch:   12/200   Loss: 5.655821352005005
Epoch:   12/200   Loss: 5.654545440673828
Epoch:   12/200   Loss: 5.659784846305847
Epoch:   12/200   Loss: 5.684938073158264
Epoch:   12/200   Loss: 5.701318860054016
Epoch:   12/200   Loss: 5.659614868164063
Epoch:   12/200   Loss: 5.671294713020325
Epoch:   12/200   Loss: 5.677239603996277
Epoch:   12/200   Loss: 5.6418816614151
Epoch:   12/200   Loss: 5.641601271629334
Epoch:   12/200   Loss: 5.6544276714324955
Epoch:   12/200   Loss: 5.702397751808166
Epoch:   12/200   Loss: 5.651774082183838
Epoch:   12/200   Loss: 5.681333045959473
Epoch:   13/200   Loss: 5.660456374935482
Epoch:   13/200   Loss: 5.6355928039550784
Epoch:   13/200   Loss: 5.619849152565003
Epoch:   13/200   Loss: 5.644858264923096
Epoch:   13/200   Loss: 5.647326321601867
Epoch:   13/200   Loss: 5.6566130638

Epoch:   18/200   Loss: 5.664154376983642
Epoch:   18/200   Loss: 5.656018576622009
Epoch:   18/200   Loss: 5.648664960861206
Epoch:   18/200   Loss: 5.661584696769714
Epoch:   18/200   Loss: 5.654688754081726
Epoch:   18/200   Loss: 5.67311056137085
Epoch:   18/200   Loss: 5.625758061408996
Epoch:   18/200   Loss: 5.628401765823364
Epoch:   18/200   Loss: 5.647979106903076
Epoch:   18/200   Loss: 5.64411880493164
Epoch:   18/200   Loss: 5.673836250305175
Epoch:   18/200   Loss: 5.660435848236084
Epoch:   18/200   Loss: 5.670503253936768
Epoch:   18/200   Loss: 5.650693688392639
Epoch:   18/200   Loss: 5.682005233764649
Epoch:   18/200   Loss: 5.679923586845398
Epoch:   18/200   Loss: 5.667178649902343
Epoch:   18/200   Loss: 5.698080010414124
Epoch:   18/200   Loss: 5.663555331230164
Epoch:   18/200   Loss: 5.67422733783722
Epoch:   18/200   Loss: 5.654561986923218
Epoch:   18/200   Loss: 5.661434926986694
Epoch:   18/200   Loss: 5.668879113197327
Epoch:   18/200   Loss: 5.64631906032

Epoch:   23/200   Loss: 5.626843481063843
Epoch:   24/200   Loss: 5.647082678649737
Epoch:   24/200   Loss: 5.640191693305969
Epoch:   24/200   Loss: 5.643282279968262
Epoch:   24/200   Loss: 5.645940566062928
Epoch:   24/200   Loss: 5.6371931552886965
Epoch:   24/200   Loss: 5.6612201595306395
Epoch:   24/200   Loss: 5.657514843940735
Epoch:   24/200   Loss: 5.668146333694458
Epoch:   24/200   Loss: 5.665084896087646
Epoch:   24/200   Loss: 5.66709135055542
Epoch:   24/200   Loss: 5.666271810531616
Epoch:   24/200   Loss: 5.645459127426148
Epoch:   24/200   Loss: 5.630151844024658
Epoch:   24/200   Loss: 5.646379251480102
Epoch:   24/200   Loss: 5.6681119203567505
Epoch:   24/200   Loss: 5.666731276512146
Epoch:   24/200   Loss: 5.64546886920929
Epoch:   24/200   Loss: 5.65753809928894
Epoch:   24/200   Loss: 5.629607481956482
Epoch:   24/200   Loss: 5.648289351463318
Epoch:   24/200   Loss: 5.661583094596863
Epoch:   24/200   Loss: 5.678124709129333
Epoch:   24/200   Loss: 5.68977606

...

Epoch:  173/200   Loss: 5.676515355110168
Epoch:  173/200   Loss: 5.631028022766113
Epoch:  173/200   Loss: 5.637513880729675
Epoch:  173/200   Loss: 5.652764358520508
Epoch:  173/200   Loss: 5.6584547328948975
Epoch:  173/200   Loss: 5.660876288414001
Epoch:  173/200   Loss: 5.660917463302613
Epoch:  173/200   Loss: 5.697703504562378
Epoch:  173/200   Loss: 5.660848784446716
Epoch:  173/200   Loss: 5.64589828491211
Epoch:  173/200   Loss: 5.661459584236145
Epoch:  173/200   Loss: 5.666696944236755
Epoch:  173/200   Loss: 5.681605820655823
Epoch:  173/200   Loss: 5.6526096200942995
Epoch:  173/200   Loss: 5.637863073348999
Epoch:  173/200   Loss: 5.6820982074737545
Epoch:  174/200   Loss: 5.642526090145111
Epoch:  174/200   Loss: 5.647619500160217
Epoch:  174/200   Loss: 5.6610484170913695
Epoch:  174/200   Loss: 5.658692851066589
Epoch:  174/200   Loss: 5.662688136100769
Epoch:  174/200   Loss: 5.636922731399536
Epoch:  174/200   Loss: 5.641607456207275
Epoch:  174/200   Loss: 5.67025

Epoch:  179/200   Loss: 5.683020648956298
Epoch:  179/200   Loss: 5.6593597841262815
Epoch:  179/200   Loss: 5.672189435958862
Epoch:  179/200   Loss: 5.642076196670533
Epoch:  179/200   Loss: 5.625854063034057
Epoch:  179/200   Loss: 5.650257844924926
Epoch:  179/200   Loss: 5.622837061882019
Epoch:  179/200   Loss: 5.654337363243103
Epoch:  179/200   Loss: 5.6343844747543335
Epoch:  179/200   Loss: 5.666834440231323
Epoch:  179/200   Loss: 5.65422037601471
Epoch:  179/200   Loss: 5.627411627769471
Epoch:  179/200   Loss: 5.6722062301635745
Epoch:  179/200   Loss: 5.64940423488617
Epoch:  179/200   Loss: 5.650780034065247
Epoch:  179/200   Loss: 5.671642003059387
Epoch:  179/200   Loss: 5.653049864768982
Epoch:  179/200   Loss: 5.609483060836792
Epoch:  179/200   Loss: 5.628744025230407
Epoch:  179/200   Loss: 5.671054248809814
Epoch:  179/200   Loss: 5.6939386367797855
Epoch:  179/200   Loss: 5.632568874359131
Epoch:  179/200   Loss: 5.6512306213378904
Epoch:  179/200   Loss: 5.68420

Epoch:  185/200   Loss: 5.660279253254766
Epoch:  185/200   Loss: 5.629234237670898
Epoch:  185/200   Loss: 5.669506230354309
Epoch:  185/200   Loss: 5.644795951843261
Epoch:  185/200   Loss: 5.637443723678589
Epoch:  185/200   Loss: 5.643309063911438
Epoch:  185/200   Loss: 5.651205220222473
Epoch:  185/200   Loss: 5.663551998138428
Epoch:  185/200   Loss: 5.6704559946060185
Epoch:  185/200   Loss: 5.642353563308716
Epoch:  185/200   Loss: 5.640721936225891
Epoch:  185/200   Loss: 5.679032835960388
Epoch:  185/200   Loss: 5.666586203575134
Epoch:  185/200   Loss: 5.640038681030274
Epoch:  185/200   Loss: 5.657188639640808
Epoch:  185/200   Loss: 5.650807251930237
Epoch:  185/200   Loss: 5.655315666198731
Epoch:  185/200   Loss: 5.680098171234131
Epoch:  185/200   Loss: 5.611038718223572
Epoch:  185/200   Loss: 5.628289546966553
Epoch:  185/200   Loss: 5.642552409172058
Epoch:  185/200   Loss: 5.662466907501221
Epoch:  185/200   Loss: 5.642723579406738
Epoch:  185/200   Loss: 5.6838472

Epoch:  190/200   Loss: 5.674117269515992
Epoch:  190/200   Loss: 5.649559497833252
Epoch:  190/200   Loss: 5.634485211372375
Epoch:  190/200   Loss: 5.6507144546508785
Epoch:  190/200   Loss: 5.688100771903992
Epoch:  190/200   Loss: 5.651821670532226
Epoch:  190/200   Loss: 5.658081922531128
Epoch:  190/200   Loss: 5.658735814094544
Epoch:  190/200   Loss: 5.68214448928833
Epoch:  191/200   Loss: 5.641608450723731
Epoch:  191/200   Loss: 5.607465057373047
Epoch:  191/200   Loss: 5.64085506439209
Epoch:  191/200   Loss: 5.6716491317749025
Epoch:  191/200   Loss: 5.676510753631592
Epoch:  191/200   Loss: 5.651317443847656
Epoch:  191/200   Loss: 5.660303826332092
Epoch:  191/200   Loss: 5.676473684310913
Epoch:  191/200   Loss: 5.68475944519043
Epoch:  191/200   Loss: 5.631935753822327
Epoch:  191/200   Loss: 5.628578624725342
Epoch:  191/200   Loss: 5.6590927219390865
Epoch:  191/200   Loss: 5.6448077917098995
Epoch:  191/200   Loss: 5.640594682693481
Epoch:  191/200   Loss: 5.6349759

Epoch:  196/200   Loss: 5.623727102279663
Epoch:  196/200   Loss: 5.673192324638367
Epoch:  196/200   Loss: 5.633445801734925
Epoch:  196/200   Loss: 5.656492881774902
Epoch:  196/200   Loss: 5.657295174598694
Epoch:  196/200   Loss: 5.6883594512939455
Epoch:  196/200   Loss: 5.654760017395019
Epoch:  196/200   Loss: 5.651019024848938
Epoch:  196/200   Loss: 5.655692977905273
Epoch:  196/200   Loss: 5.66941077709198
Epoch:  196/200   Loss: 5.685202422142029
Epoch:  196/200   Loss: 5.646934962272644
Epoch:  196/200   Loss: 5.677907085418701
Epoch:  196/200   Loss: 5.650041480064392
Epoch:  196/200   Loss: 5.642426781654358
Epoch:  196/200   Loss: 5.675685091018677
Epoch:  196/200   Loss: 5.657089600563049
Epoch:  196/200   Loss: 5.630241136550904
Epoch:  197/200   Loss: 5.675514114939648
Epoch:  197/200   Loss: 5.633696546554566
Epoch:  197/200   Loss: 5.6633575868606565
Epoch:  197/200   Loss: 5.645399522781372
Epoch:  197/200   Loss: 5.669354972839355
Epoch:  197/200   Loss: 5.6687502

In [50]:
# Number of Epochs
num_epochs = 1
# Learning Rate
learning_rate = 0.01
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Number of RNN Layers
n_layers = 3
# Show stats for every n number of batches
show_every_n_batches = 200
# ===========================================

# of words in a sequence
sequence_lengths = [16]
# Batch Size
batch_sizes = [128]
# Embedding Dimension
embedding_dims = [32, 512]
# Hidden Dimension
hidden_dims = [32, 512]

for sequence_length in sequence_lengths:
    for batch_size in batch_sizes:
        for embedding_dim in embedding_dims:
            for hidden_dim in hidden_dims:
                print('========================================================================================')
                print('sequence_length:', sequence_length)
                print('batch_size:', batch_size)
                print('embedding_dim:', embedding_dim)
                print('hidden_dim:', hidden_dim)
                print('')
                # data loader - do not change
                train_loader = batch_data(int_text, sequence_length, batch_size)

                # create model and move to gpu if available
                rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
                if train_on_gpu:
                    rnn.cuda()

                # defining loss and optimization functions for training
                optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
                criterion = nn.CrossEntropyLoss()

                # training the model
                trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

sequence_length: 16
batch_size: 128
embedding_dim: 32
hidden_dim: 32

Training for 1 epoch(s)...
Epoch:    1/1     Loss: 6.670435824394226
Epoch:    1/1     Loss: 5.782892792224884
Epoch:    1/1     Loss: 5.766704573631286
Epoch:    1/1     Loss: 5.741564931869507
Epoch:    1/1     Loss: 5.770444393157959
Epoch:    1/1     Loss: 5.761946439743042
Epoch:    1/1     Loss: 5.739880349636078
Epoch:    1/1     Loss: 5.750335910320282
Epoch:    1/1     Loss: 5.721570808887481
Epoch:    1/1     Loss: 5.743580231666565
Epoch:    1/1     Loss: 5.759668526649475
Epoch:    1/1     Loss: 5.708649804592133
Epoch:    1/1     Loss: 5.780300827026367
Epoch:    1/1     Loss: 5.765619292259216
Epoch:    1/1     Loss: 5.70572999715805
Epoch:    1/1     Loss: 5.6997741603851315
Epoch:    1/1     Loss: 5.696344652175903
Epoch:    1/1     Loss: 5.712625844478607
Epoch:    1/1     Loss: 5.746235358715057
Epoch:    1/1     Loss: 5.753505043983459
Epoch:    1/1     Loss: 5.734697825908661
Epoch:    1/1     Los

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:**

At the beginnign I just started with random values for sequence_length, batch_size, hidden_dim and embedding_dim. learning_rate was 0.001. I did not want to put too high learning rate to see what accuraccy I can reach with 10 training epochs. I used LSTM for RNN with 3 layers (reason for 3 layers comes frmo rubric where recommended number of layers is between 1 and 3).

Aleady this network showed very low loss value. But the generated text was very, very bad.

Afterwards I decided to check the performance of the networks with other parameters. So I wrote the for loop and tested the accuracy of the network with higher learning rate (0.01) and lower/higher embedding dim and hidden dim, I also checked if we can improve the accuracy by increasing the number of layers.

The result was the same. Accuraccy was very low but generated text was terrible.

I also decided to check if we can improvee the generated text quality with another parameters for sequence length and batch size. No improvements, and by sequence_length: 64 batch_size: 256 I was out of the memory on my GPU.

Looks lie for improving the quality of the text we need to do data pre-processing and increase the ammount of the data.

But I can conclude on the available amount of data almost any configuration of the network will provide good loss and terrible text.

**Update after firat submission**

After the first submission I removed the dropout adter the LSTM and also added one more fully connected layer. Also added the normalization between the FC layers.

I noticed, if I remove the sigmoid at the end I get the negative loss, I I left the sigmoid.

As the result I still have the same very low accuraccy and very poor quality of the text.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [102]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [103]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [104]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry:
?,

.. you
,,,..?.


?.,.



.
.

,?

you,..?
.
,
you


,

you?.,
,.



.,
.,,

.
??,
.,


?...
, you

.,.
.,,,



.



,
you

.,, you..?
,??. you.,

..


?
.
, you
.
.?,.
.
.
...





,

,

you.
...
. you you

.,. you..
,

.
.?

,..



,.
you?



?


..
?.,?.,,
you
.?



?


.,,. you
.
. you,,
?
..,..,,


,
..

,.
?
.
.....

.



you
..?
you??
you you?.,.,
?
.,



you
.
..,.,.,.,
. you,.,,



,
you you
you


.?,



. you
.?.
.

,,
.,

.,.


,, you?

,

.
.
.

.,
,.


#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [105]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.